In [ ]:
input_dir = "input"

In [ ]:
# add project root to module path if not added already
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path and os.path.basename(module_path) == "t212-tax-and-profit":
  sys.path.append(module_path)
  print(f"Path '{module_path}' added to module path.")


In [ ]:
from services.csv_consolidator import consolidate

events = consolidate(input_dir)

In [ ]:
from services.event_emitter import EventEmitter
from services.event_processors import DepositWithdrawalProcessor, DividendProcessor
from models.input import Action

event_emitter: EventEmitter = EventEmitter()

deposit_withdrawal_processor = DepositWithdrawalProcessor()
event_emitter.subscribe(deposit_withdrawal_processor, [Action.DEPOSIT, Action.WITHDRAWAL])

dividend_processor = DividendProcessor()
event_emitter.subscribe(dividend_processor, Action.DIVIDEND)

event_emitter.start(events)

### Profit calculation

### Dividend calculation

In [ ]:
dividends: list[DividendProcessor.Dividend] = dividend_processor.result()
total_dividend = sum([d.dividend for d in dividends])
total_withholding_tax = sum([d.withholding_tax for d in dividends])
print(f"Total dividends: {total_dividend:.2f}")
print(f"Total withholding tax: {total_withholding_tax:.2f}")

In [ ]:
for d in dividends:
    print(d)

In [ ]:
import plotly.graph_objects as go

header_values = ['Name', 'Ticker', 'Dividend', 'Tax', 'Tax currency']
values = [
            [d.name for d in dividends],
            [d.ticker for d in dividends],
            [d.dividend for d in dividends],
            [d.withholding_tax for d in dividends],
            [d.withholding_tax_currency for d in dividends]
        ]

fig = go.Figure(data=[go.Table(
    header=dict(values=header_values,
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=values,
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])

fig.update_layout(width=500, height=2000)
fig.show()

### Total gains

### Total deposits and withdrawals

In [ ]:
deposit_withdrawal_result: DepositWithdrawalProcessor.Result = deposit_withdrawal_processor.result()

print(f"Total deposit: {deposit_withdrawal_result.deposit:.2f}")
print(f"Total deposit fees: {deposit_withdrawal_result.deposit_fee}")
print(f"Total withdrawals: {deposit_withdrawal_result.withdrawal:.2f}")